In [1]:
import os
import re
import pickle
import concurrent.futures
from glob import glob

from tqdm import tqdm

from KeywordSearch import loader
from KeywordSearch.utils import construct_bool_table
from KeywordSearch.cloud_index import prepare_tokendict_for_upload, upload_firestore

Please ignore the syntax warnings as small integers in CPython are singletons
Using `is` instead of `=` for comparison in performance-critical code is acceptable


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10022\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from google.cloud import firestore
db = firestore.Client(project="moonlit-oven-412316")
index_api = db.collection("index")

In [13]:
file_str = '''import os
import re
import pickle
import concurrent.futures
from glob import glob

from tqdm import tqdm

from KeywordSearch import loader
from KeywordSearch.cloud_index import prepare_tokendict_for_upload, upload_firestore

from google.cloud import firestore
db = firestore.Client(project="moonlit-oven-412316")
index_api = db.collection("index")

pickled_segments = glob("*_merged.pkl", root_dir=loader.index_dir)
regex_segment_id = re.compile("([0-9]+)_merged.pkl")
checkpoint = %d
stopat = %d

num_segments = len(pickled_segments)
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as pool:
    with tqdm(total=(stopat-checkpoint)*5000, desc="Initializing", ncols=60, ascii=True) as pbar:
        for counter, fname in enumerate(pickled_segments, 1):
            segment_id = int(regex_segment_id.fullmatch(fname).group(1))
            if segment_id < checkpoint or segment_id >= stopat:
                continue
            offset = 5000 * segment_id
            with open(os.path.join(loader.index_dir, fname), "rb") as f:
                current_slice = pickle.load(f)
            pbar.set_description(f"Segment {counter:4d}/{num_segments:4d}")
            jobs = {pool.submit(prepare_tokendict_for_upload, token_dict, i) : i 
                    for i, token_dict in enumerate(current_slice, start=offset)}
            for i, job in enumerate(concurrent.futures.as_completed(jobs)):
                token_id = jobs[job]
                for doc in job.result():
                    upload_firestore(doc, index_api)
                if i % 100 is 0:
                    pbar.update(100)'''

range_str = '''checkpoint = %d
stopat = %d'''

for i, checkpoint in enumerate(range(0, 2800, 200)):
    stopat = checkpoint + 200
    with open(f"u{i}", 'w', encoding="utf-8") as f:
        f.write(file_str.replace(range_str, range_str %(checkpoint, stopat)))
    with open(f"run{i}.bat", 'w', encoding="utf-8") as f:
        f.write(f"python u{i}\npause\n")

i += 1
with open(f"u{i}", 'w', encoding="utf-8") as f:
    f.write(file_str.replace(range_str, range_str %(2800, -1)).replace(" or segment_id >= stopat", '').replace("(stopat-checkpoint)*5000", "len(loader.all_tokens)-(checkpoint*5000)"))
with open(f"run{i}.bat", 'w', encoding="utf-8") as f:
    f.write(f"python u{i}\npause\n")


In [3]:
# pickled_segments = glob("*_merged.pkl", root_dir=loader.index_dir)
# regex_segment_id = re.compile("([0-9]+)_merged.pkl")
# checkpoint = 122
# stopat=400

In [4]:
# num_segments = len(pickled_segments)
# with concurrent.futures.ThreadPoolExecutor() as pool:
#     with tqdm(total=(stopat-checkpoint)*5000, desc="Initializing", ncols=120, ascii=True) as pbar:
#         for counter, fname in enumerate(pickled_segments, 1):
#             segment_id = int(regex_segment_id.fullmatch(fname).group(1))
#             if segment_id < checkpoint or segment_id >= stopat:
#                 continue
#             offset = 5000 * segment_id
#             with open(os.path.join(loader.index_dir, fname), "rb") as f:
#                 current_slice = pickle.load(f)
#             pbar.set_description(f"Segment {counter:4d}/{num_segments:4d}")
#             jobs = {pool.submit(prepare_tokendict_for_upload, token_dict, i) : i 
#                     for i, token_dict in enumerate(current_slice, start=offset)}
#             for i, job in enumerate(concurrent.futures.as_completed(jobs)):
#                 token_id = jobs[job]
#                 for doc in job.result():
#                     upload_firestore(doc, index_api)
#                 if i % 100 is 0:
#                     pbar.update(100)

<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\10022\AppData\Local\Temp\ipykernel_17156\2593883698.py:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i % 100 is 0:
Segment  400/2987: 100%|###################################################| 1390000/1390000 [12:41:47<00:00, 30.41it/s]
